<a href="https://colab.research.google.com/github/Sudheer-Arora/Intent-Recognition-using-TensorFlow/blob/main/Load_model_into_new_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import json


In [16]:
from tensorflow.keras.models import load_model
loaded_model = load_model('intent_classification_model.h5')
print("Model loaded successfully!")

Model loaded successfully!


In [17]:
loaded_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 9, 300)              │          39,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 100)                 │         140,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 23)                  │           1,173 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 186,225 (727.45 KB)

 Trainable params: 186,223 (727.43 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [18]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
print("Tokenizer loaded successfully!")

Tokenizer loaded successfully!


In [19]:
with open('response_for_intent.json', 'r') as f:
    response_for_intent = json.load(f)
print("Response map loaded successfully!")

Response map loaded successfully!


In [20]:
with open('index_to_intent.json', 'r') as f:
    index_to_intent = json.load(f)
print("Index-to-intent mapping loaded successfully!")

Index-to-intent mapping loaded successfully!


In [21]:
def response(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index['<unk>'])
    # Pad the sequence to match the model's input length
    sent_tokens = pad_sequences([sent_tokens], padding='pre', maxlen=20)  # Adjust maxlen based on training
    # Predict the intent
    pred = loaded_model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    print(f"Predicted class index: {pred_class[0]}")  # Debugging line
    # Safely get the intent
    print(f"Index-to-intent mapping: {index_to_intent}")  # Debugging line

    # Check if the predicted class index exists in the index_to_intent dictionary
    if str(pred_class[0]) in index_to_intent:  # Convert pred_class[0] to string for key lookup
        predicted_intent = index_to_intent[str(pred_class[0])]  # Access using string key
    else:
        # Handle the case where the predicted class index is not found
        print(f"Warning: Predicted class index {pred_class[0]} not found in index_to_intent.")
        predicted_intent = "Unknown"  # Or handle it differently based on your logic

    if predicted_intent == "Unknown":
        return "I'm sorry, I didn't understand that.", predicted_intent
    # Return a random response for the predicted intent
    return random.choice(response_for_intent[predicted_intent]), predicted_intent

In [22]:
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print('Geek: {} -- TYPE: {}'.format(bot_response, typ))
    print()

Note: Enter 'quit' to break the loop.
You: hello
Predicted class index: 0
Index-to-intent mapping: {'0': 'Greeting', '1': 'GreetingResponse', '2': 'FavoriteFood', '3': 'CourtesyGreeting', '4': 'CourtesyGreetingResponse', '5': 'CurrentHumanQuery', '6': 'NameQuery', '7': 'RealNameQuery', '8': 'TimeQuery', '9': 'Thanks', '10': 'NotTalking2U', '11': 'UnderstandQuery', '12': 'Shutup', '13': 'Swearing', '14': 'GoodBye', '15': 'CourtesyGoodBye', '16': 'WhoAmI', '17': 'Clever', '18': 'Gossip', '19': 'Jokes', '20': 'PodBayDoor', '21': 'PodBayDoorResponse', '22': 'SelfAware'}
Geek: Hola human, please tell me your GeniSys user -- TYPE: Greeting

You: your favorite food
Predicted class index: 2
Index-to-intent mapping: {'0': 'Greeting', '1': 'GreetingResponse', '2': 'FavoriteFood', '3': 'CourtesyGreeting', '4': 'CourtesyGreetingResponse', '5': 'CurrentHumanQuery', '6': 'NameQuery', '7': 'RealNameQuery', '8': 'TimeQuery', '9': 'Thanks', '10': 'NotTalking2U', '11': 'UnderstandQuery', '12': 'Shutup',